## Creating binary models vs full grade levels

In [1]:
from article_process import ArticleLM

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import pandas as pd

In [3]:
path_to_data = '/Users/stephanie/data/newsela_article_corpus_2016-01-29'
path_to_kenlm = '/Users/stephanie/github/kenlm'
path_to_arpa = path_to_kenlm + '/lm'

In [4]:
# pd.read_csv('/Users/stephanie/data/newsela_articles_corpus_2019-03-25/MANIFEST.csv')

In [4]:
articleLM = ArticleLM(path_to_data, path_to_kenlm, path_to_arpa, 5, 'binary', True)

In [6]:
# articleLM.metadata_split

## Builds the processed sentences that I need for KenLM

In [5]:
articleLM.build_data()

In [36]:
articleLM.train_all_arpas()

Processing hard
Grade hard has 243562 sentences in the train set.
=== 1/5 Counting and sorting n-grams ===
File stdin isn't normal.  Using slower read() instead of mmap().  No progress bar.
Unigram tokens 4764661 types 60616
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:727392 2:1340796800 3:2513994240 4:4022390272 5:5865986048
Statistics:
1 60616 D1=0.573557 D2=1.01017 D3+=1.51199
2 789627 D1=0.714274 D2=1.175 D3+=1.52528
3 1913778 D1=0.820462 D2=1.29831 D3+=1.62168
4 2537234 D1=0.882494 D2=1.41292 D3+=1.93552
5 2713137 D1=0.564295 D2=1.15154 D3+=2.75385
Memory estimate for binary LM:
type     MB
probing 168 assuming -p 1.5
probing 198 assuming -r models -p 1.5
trie     79 without quantization
trie     42 assuming -q 8 -b 8 quantization 
trie     70 assuming -a 22 array pointer compression
trie     33 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:727392 2:12634032 3:3827

## Printing out stats

how many sentences there are in training set, indexed by grade level?

In [7]:
for g in articleLM.level_sentences.get('train').keys():
    print(g, len(articleLM.level_sentences.get('train').get(g)))

hard 243562
easy 158136


In [8]:
# Number of sentences, train
sentence_len = 0
for g in articleLM.level_sentences.get('train').keys():
    sentence_len+=len(articleLM.level_sentences.get('train').get(g))
    
print(sentence_len)

401698


In [9]:
# Number of sentences, validation
sentence_len = 0
for g in articleLM.level_sentences.get('val').keys():
    sentence_len+=len(articleLM.level_sentences.get('val').get(g))
print(sentence_len)

51067


how many articles there are in each set, indexed by grade level?

In [10]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='train'].grade_level.value_counts()

12    1660
4     1399
5     1167
7     1092
6      943
8      922
9      699
3      503
2      225
11       2
Name: grade_level, dtype: int64

In [33]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='train'].grade_level.value_counts().sum()

8612

In [34]:
articleLM.metadata_split[articleLM.metadata_split.train_val_test=='val'].grade_level.value_counts()

12    205
4     166
5     151
7     150
6     120
8     115
9      88
3      55
2      26
Name: grade_level, dtype: int64

## Comparing perplexities - Sentence Level

In [11]:
perplex_guesses = articleLM.compute_all_sentences_best_guess(['easy', 'hard'])

In [12]:
len(perplex_guesses[perplex_guesses.level == perplex_guesses.best_guess] ) * 1. / len(perplex_guesses)

0.7075606556092976

## Article level

In [14]:
# def compute_article_best_guess():
#     article_sentences = articleLM.article_val_sentences

#     iter = 0
#     article_df = pd.DataFrame({})
#     for article_gl in article_sentences.keys():
#         #     mean_guess, median_guess = articleLM.compute_perplexity_entire_article(article_sentences[article_gl])

#         sample_perp = dict()
#         for ix, sentence in enumerate(article_sentences[article_gl]):
#             perplexities = articleLM.compute_sentence_perplexities(sentence)
#             sample_perp[ix] = perplexities
#         sample_perp_df = pd.DataFrame(sample_perp).T
#         sample_perp_df.columns = sorted(articleLM.models.keys())

#         sample_perp_df.loc[:, 'min_perplexity'] = sample_perp_df.idxmin(axis=1, skipna=True)
#         sample_perp_df.loc[:, 'true_gl'] = article_gl[1]
#         sample_perp_df.loc[:, 'article'] = article_gl[0]

#         article_df = pd.concat([article_df, sample_perp_df])

#         iter += 1

#         if iter % 10 == 0:
#             print("iteration {}, article {}".format(iter, article_gl))

#     return article_df

# # def plot_best_article_guess():
# #     article_df = articleLM.compute_article_best_guess()
# #     means = article_df.groupby(['article', 'true_gl']).mean()['min_perplexity'].reset_index()
# #     means.rename(columns={'min_perplexity': 'predicted_gl'}, inplace=True)

# #     fig = plt.figure(figsize=(10, 8))
# #     ax = fig.gca()
# #     ax.set_title("Grade Level Distributions")
# #     # ax.set_xlabel("True Grade Level")
# #     # ax.set_ylabel("Predicted Grade Level")
# #     sns.boxplot(ax=ax, x="true_gl", y="predicted_gl", data=means)
# #     ax.set_xlabel("True Grade Level")
# #     ax.set_ylabel("Predicted Grade Level, using Means")

# #     return article_df

In [6]:
article_df, best_guess_val = articleLM.compute_article_best_guess('val')

iteration 100, article ('blob-warmwater', 'hard')
iteration 200, article ('crabs-toxic', 'hard')
iteration 300, article ('faster-baseball', 'hard')
iteration 400, article ('inuit-food', 'hard')
iteration 500, article ('minoritygap-youngmen', 'easy')
iteration 600, article ('parks-lines', 'hard')
iteration 700, article ('sharks-hawaii', 'hard')
iteration 800, article ('trophyhunting-research', 'easy')


In [7]:
best_guess_val

,article,true_gl,easy,hard,predicted_gl
0,10dollarbill-woman,hard,373.236355,21.183493,hard
1,20dollarbill-female,hard,16.998070,10.734050,hard
2,3d-indoormap,hard,62.048541,175.140802,hard
3,401k-literacy,hard,523.651247,15.684725,hard
4,Braille-startup,hard,244.302287,3.531025,hard
5,Civilrights-comicbook,hard,30.740547,4.111333,hard
6,D-day-fallen,easy,23.147835,20.448213,easy
7,Detroit-bankruptcy,easy,34.207341,18.075680,easy
8,Fastfood-calories,hard,316.192900,4.509618,hard
9,Hospital-bootcamp,hard,4.203880,56.149446,easy


In [8]:
(cm, binary_accuracy, binary_f1) = articleLM.compute_scores_binary(best_guess_val)

In [9]:
cm

array([[180, 153],
       [ 24, 516]])

In [10]:
binary_accuracy

0.7972508591065293

In [11]:
binary_f1

0.6703910614525139